In [1]:
import requests
import pandas as pd
import sys
from io import BytesIO
import gzip
import io
import os
from global_variables import *
import sqlite3
from urllib.parse import urlparse

In [22]:
poi_sod = pd.read_csv("C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/poi_sod_4.csv.gz", compression='gzip')

poi = pd.read_csv("C:/Users/dratnadiwakara2/Documents/OneDrive - Louisiana State University/Raw Data/Dewey/all_us_poi_with_store_id.csv.gz", compression='gzip',usecols=['PLACEKEY','POSTAL_CODE','STREET_ADDRESS','STORE_ID'])

poi_sod = pd.merge(poi, poi_sod, on='PLACEKEY', how='inner')

del poi

poi_sod['match_address'] = poi_sod['STREET_ADDRESS'].str.lower()+' '+poi_sod['POSTAL_CODE'].astype(int).astype(str)
unique_match_addresses = poi_sod['match_address'].unique()

poi_sod['POSTAL_CODE'] = poi_sod['POSTAL_CODE'].astype(int)

In [109]:
poi_sod.head()

,PLACEKEY,POSTAL_CODE,STREET_ADDRESS,STORE_ID,CERT,UNINUMBR,NAMEFULL,usps_address,full_address_sod,LOCATION_NAME,match_type,match_address
0,225-222@5vg-3sn-7yv,95688,367 Merchant St,NaN,6548,196947,U.S. Bank National Association,367 MERCHANT ST VACAVILLE CA 95688,"367 merchant street, vacaville, 95688 ca",U.S. Bank,exact,367 merchant st 95688
1,225-222@5vg-3sn-7yv,95688,367 Merchant St,NaN,6548,196947,U.S. Bank National Association,367 MERCHANT ST VACAVILLE CA 95688,"367 merchant street, vacaville, 95688 ca",U.S. Bank,exact,367 merchant st 95688
2,228-223@627-wcb-hh5,7302,123 Montgomery St,NaN,30052,635576,Blue Foundry Bank,123 MONTGOMERY ST JERSEY CITY NJ 07302,"123 montgomery st, jersey city, 7302 nj",Blue Foundry Bank,exact,123 montgomery st 7302
3,228-223@627-wcb-hh5,7302,123 Montgomery St,NaN,30052,635576,Blue Foundry Bank,123 MONTGOMERY ST JERSEY CITY NJ 07302,"123 montgomery st, jersey city, 7302 nj",Blue Foundry Bank,exact,123 montgomery st 7302
4,zzw-226@627-s57-q9f,7644,184 Essex St,206639749.0,4297,467739,"Capital One, National Association",184 ESSEX ST LODI NJ 07644,"184 essex street, lodi, 7644 nj",MoneyPass ATM,exact,184 essex st 7644


In [34]:
results = requests.get(url=Retail_Store_Visitors_API_PATH,
                           params={'page': 1,
                                   'partition_key_after': '1900-01-01', # set date value here
                               'partition_key_before': '2099-01-01'},
                           headers={'X-API-KEY': API_KEY,
                                    'accept': 'application/json'
                                   })
response_json = results.json()

In [35]:
data_path = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_banks.csv'
for link_data in response_json['download_links'][195:]:
    print(f"Downloading file {link_data['file_name']}...")
    data = requests.get(link_data['link'])
    gzip_stream = io.BytesIO(data.content)
        
    df = pd.read_csv(gzip_stream, compression='gzip', low_memory=False)
    df['match_address'] = df['STREET_ADDRESS'].str.lower()+' '+df['ZIP_CODE'].astype(str)
    
    filtered_df = df[df['match_address'].isin(unique_match_addresses)]
    
    keywords = ['bank', 'finan', 'trust', 'capital']
    mask = filtered_df['BRAND'].str.lower().str.contains('|'.join(keywords), case=False)
    
    filtered_df = filtered_df[mask]
    
    if os.path.isfile(data_path):
        header_option = False  
    else:
        header_option = True  
    
    filtered_df.to_csv(data_path, mode='a', header=header_option, index=False)     

In [36]:
ft_data = pd.read_csv(data_path, low_memory=False)


In [37]:
ft_data = ft_data.reset_index()
with gzip.open(data_path+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    ft_data.to_csv(gzipped_file, index=False)




Identify individual components

In [ ]:
from tqdm import tqdm
import json

In [ ]:
ft_data = pd.read_csv("C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_banks.csv.gz", compression='gzip')

In [92]:
ft_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653483 entries, 0 to 1653482
Data columns (total 20 columns):
 #   Column                          Non-Null Count    Dtype 
---  ------                          --------------    ----- 
 0   index                           1653483 non-null  int64 
 1   STORE_ID                        1653483 non-null  object
 2   NAME                            1653483 non-null  object
 3   BRAND                           1653483 non-null  object
 4   STREET_ADDRESS                  1653483 non-null  object
 5   CITY                            1653483 non-null  object
 6   STATE                           1653483 non-null  object
 7   ZIP_CODE                        1653483 non-null  int64 
 8   MONTH_STARTING                  1653483 non-null  object
 9   MONTH_ENDING                    1653483 non-null  object
 10  TOTAL_VISITS                    1653483 non-null  int64 
 11  VISITOR_BRAND_DESTINATIONS      1653483 non-null  object
 12  VISITOR_PSYCHO

In [99]:
VISITOR_HOUSEHOLD_INCOME_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_VISITOR_HOUSEHOLD_INCOME.csv'
VISITOR_EDUCATIONAL_ATTAINMENT_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_VISITOR_EDUCATIONAL_ATTAINMENT.csv'
VISITOR_AGE_RANGE_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_VISITOR_AGE_RANGE.csv'
VISITOR_PSYCHOGRAPHIC_PROFILES_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_VISITOR_PSYCHOGRAPHIC_PROFILES.csv'

with tqdm(total=len(ft_data)) as pbar:
  
     for index in range(0, len(ft_data)):
        t = ft_data.loc[index]

        data_string = t['VISITOR_HOUSEHOLD_INCOME'].replace('},{', '_') .replace("{{","{").replace("}}","}")
        data_string = data_string.replace(",",":").replace("_",",")
        data_dict = json.loads(data_string)
        df_VISITOR_HOUSEHOLD_INCOME = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Value']).reset_index()
        df_VISITOR_HOUSEHOLD_INCOME = df_VISITOR_HOUSEHOLD_INCOME.rename(columns={'index': 'VISITOR_HOUSEHOLD_INCOME', 'Value': 'no_visits'})
        df_VISITOR_HOUSEHOLD_INCOME['index']=t['index']
        df_VISITOR_HOUSEHOLD_INCOME['STORE_ID']=t['STORE_ID']
        
        
        data_string = t['VISITOR_EDUCATIONAL_ATTAINMENT'].replace('},{', '_') .replace("{{","{").replace("}}","}")
        data_string = data_string.replace(",",":").replace("_",",")
        data_dict = json.loads(data_string)
        df_VISITOR_EDUCATIONAL_ATTAINMENT = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Value']).reset_index()
        df_VISITOR_EDUCATIONAL_ATTAINMENT = df_VISITOR_EDUCATIONAL_ATTAINMENT.rename(columns={'index': 'VISITOR_EDUCATIONAL_ATTAINMENT', 'Value': 'no_visits'})
        df_VISITOR_EDUCATIONAL_ATTAINMENT['index']=t['index']
        df_VISITOR_EDUCATIONAL_ATTAINMENT['STORE_ID']=t['STORE_ID']
        
        
        data_string = t['VISITOR_AGE_RANGE'].replace('},{', '_') .replace("{{","{").replace("}}","}")
        data_string = data_string.replace(",",":").replace("_",",")
        data_dict = json.loads(data_string)
        df_VISITOR_AGE_RANGE = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Value']).reset_index()
        df_VISITOR_AGE_RANGE = df_VISITOR_AGE_RANGE.rename(columns={'index': 'VISITOR_AGE_RANGE', 'Value': 'no_visits'})
        df_VISITOR_AGE_RANGE['index']=t['index']
        df_VISITOR_AGE_RANGE['STORE_ID']=t['STORE_ID']
        
                
        data_string = t['VISITOR_PSYCHOGRAPHIC_PROFILES'].replace('},{', '_') .replace("{{","{").replace("}}","}")
        data_string = data_string.replace(",",":").replace("_",",")
        data_dict = json.loads(data_string)
        df_VISITOR_PSYCHOGRAPHIC_PROFILES = pd.DataFrame.from_dict(data_dict, orient='index', columns=['Value']).reset_index()
        df_VISITOR_PSYCHOGRAPHIC_PROFILES = df_VISITOR_PSYCHOGRAPHIC_PROFILES.rename(columns={'index': 'VISITOR_PSYCHOGRAPHIC_PROFILES', 'Value': 'no_visits'})
        df_VISITOR_PSYCHOGRAPHIC_PROFILES['NewColumn'] = df_VISITOR_PSYCHOGRAPHIC_PROFILES['VISITOR_PSYCHOGRAPHIC_PROFILES'].str.extract('(\d+)')
        df_VISITOR_PSYCHOGRAPHIC_PROFILES['NewColumn'] = pd.to_numeric(df_VISITOR_PSYCHOGRAPHIC_PROFILES['NewColumn'])
        df_VISITOR_PSYCHOGRAPHIC_PROFILES = df_VISITOR_PSYCHOGRAPHIC_PROFILES.groupby('NewColumn')['no_visits'].sum().reset_index()
        df_VISITOR_PSYCHOGRAPHIC_PROFILES = df_VISITOR_PSYCHOGRAPHIC_PROFILES.rename(columns={'NewColumn': 'VISITOR_PSYCHOGRAPHIC_PROFILES', 'no_visits': 'no_visits'})
        df_VISITOR_PSYCHOGRAPHIC_PROFILES['index']=t['index']
        df_VISITOR_PSYCHOGRAPHIC_PROFILES['STORE_ID']=t['STORE_ID']
        
        
        if os.path.isfile(VISITOR_HOUSEHOLD_INCOME_file):
            header_option = False  
        else:
            header_option = True
            
        df_VISITOR_HOUSEHOLD_INCOME.to_csv(VISITOR_HOUSEHOLD_INCOME_file, index=False, mode='a', header=header_option,sep="|")
        
        df_VISITOR_EDUCATIONAL_ATTAINMENT.to_csv(VISITOR_EDUCATIONAL_ATTAINMENT_file, index=False, mode='a', header=header_option,sep="|")
        
        df_VISITOR_AGE_RANGE.to_csv(VISITOR_AGE_RANGE_file, index=False, mode='a', header=header_option,sep="|")
         
        df_VISITOR_PSYCHOGRAPHIC_PROFILES.to_csv(VISITOR_PSYCHOGRAPHIC_PROFILES_file, index=False, mode='a', header=header_option,sep="|")
       
        pbar.update(1)

100%|██████████| 1653483/1653483 [5:41:30<00:00, 80.69it/s]  


In [132]:
df_VISITOR_HOUSEHOLD_INCOME = pd.read_csv(VISITOR_HOUSEHOLD_INCOME_file)
with gzip.open(VISITOR_HOUSEHOLD_INCOME_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    df_VISITOR_HOUSEHOLD_INCOME.to_csv(gzipped_file, index=False,sep="|")
    
df_VISITOR_EDUCATIONAL_ATTAINMENT = pd.read_csv(VISITOR_EDUCATIONAL_ATTAINMENT_file)
with gzip.open(VISITOR_EDUCATIONAL_ATTAINMENT_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    df_VISITOR_EDUCATIONAL_ATTAINMENT.to_csv(gzipped_file, index=False,sep="|")

df_VISITOR_PSYCHOGRAPHIC_PROFILES = pd.read_csv(VISITOR_PSYCHOGRAPHIC_PROFILES_file)
with gzip.open(VISITOR_PSYCHOGRAPHIC_PROFILES_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    df_VISITOR_PSYCHOGRAPHIC_PROFILES.to_csv(gzipped_file, index=False,sep="|")

In [133]:
df_VISITOR_AGE_RANGE = pd.read_csv(VISITOR_AGE_RANGE_file,sep="|")
with gzip.open(VISITOR_AGE_RANGE_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    df_VISITOR_AGE_RANGE.to_csv(gzipped_file, index=False,sep="|")

In [100]:
ft_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1653483 entries, 0 to 1653482
Data columns (total 20 columns):
 #   Column                          Non-Null Count    Dtype 
---  ------                          --------------    ----- 
 0   index                           1653483 non-null  int64 
 1   STORE_ID                        1653483 non-null  object
 2   NAME                            1653483 non-null  object
 3   BRAND                           1653483 non-null  object
 4   STREET_ADDRESS                  1653483 non-null  object
 5   CITY                            1653483 non-null  object
 6   STATE                           1653483 non-null  object
 7   ZIP_CODE                        1653483 non-null  int64 
 8   MONTH_STARTING                  1653483 non-null  object
 9   MONTH_ENDING                    1653483 non-null  object
 10  TOTAL_VISITS                    1653483 non-null  int64 
 11  VISITOR_BRAND_DESTINATIONS      1653483 non-null  object
 12  VISITOR_PSYCHO

In [113]:
locations = ft_data[['STORE_ID','NAME','STREET_ADDRESS','ZIP_CODE']]
locations = locations.drop_duplicates()
locations = locations.reset_index()
locations.info()
locations['STORE_ID'].nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36435 entries, 0 to 36434
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           36435 non-null  int64 
 1   STORE_ID        36435 non-null  object
 2   NAME            36435 non-null  object
 3   STREET_ADDRESS  36435 non-null  object
 4   ZIP_CODE        36435 non-null  int64 
dtypes: int64(2), object(3)
memory usage: 1.4+ MB


In [115]:
locations = pd.merge(locations,
                     poi_sod[['PLACEKEY','POSTAL_CODE','STREET_ADDRESS','LOCATION_NAME']],
                     left_on=['NAME','STREET_ADDRESS','ZIP_CODE'],
                     right_on=['LOCATION_NAME','STREET_ADDRESS','POSTAL_CODE'],
                     how='inner')
locations = locations.drop_duplicates()
locations.info()
locations['STORE_ID'].nunique()
locations['PLACEKEY'].nunique()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 71610 entries, 0 to 71609
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   index           71610 non-null  int64 
 1   STORE_ID        71610 non-null  object
 2   NAME            71610 non-null  object
 3   STREET_ADDRESS  71610 non-null  object
 4   ZIP_CODE        71610 non-null  int64 
 5   PLACEKEY        71610 non-null  object
 6   POSTAL_CODE     71610 non-null  int32 
 7   LOCATION_NAME   71610 non-null  object
dtypes: int32(1), int64(2), object(5)
memory usage: 4.1+ MB


35757

In [126]:
visits = ft_data[['index','STORE_ID','MONTH_STARTING','MONTH_ENDING','TOTAL_VISITS']]
visits.loc[:,'MONTH_STARTING'] = visits['MONTH_STARTING'].str[:10]
visits.loc[:,'MONTH_ENDING'] = visits['MONTH_ENDING'].str[:10]
visits.head()

,index,STORE_ID,MONTH_STARTING,MONTH_ENDING,TOTAL_VISITS
0,0,20a64007-7c11-40d0-b177-a85148bfc0a4,2019-01-01,2019-01-31,1672
1,1,19d40e03-7c3c-4e8b-81f6-dda30a34e6a6,2019-01-01,2019-01-31,1993
2,2,20aee2a2-79f7-4304-bc71-5dbcc6d5e6dc,2019-01-01,2019-01-31,1658
3,3,1d020057-daf9-4b31-8c7d-a43e19ceaf77,2019-01-01,2019-01-31,3594
4,4,240a364e-2e30-42f2-9e6d-eb036b1eef12,2019-01-01,2019-01-31,1916


In [130]:

locations_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_locations.csv'
visits_file = 'C:/Users/dratnadiwakara2/Downloads/temp_dewey/retail_store_visitor_visits.csv'

with gzip.open(locations_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    locations.to_csv(gzipped_file, index=False,sep="|")
    
with gzip.open(visits_file+'.gz', 'wt', encoding='utf-8') as gzipped_file:
    visits.to_csv(gzipped_file, index=False,sep="|")